# Lab Solutions: APIs

##### Instructions: 

Write your own Python script to answer the following questions: 
1. Which of these embassies is closest to the White House in meters? What is the address? 
2. If I wanted to hold a morning meeting there, which cafe would you suggest (best rating and closest)?
3. If I wanted to hold an upscale evening meeting there, which fancy bar would you suggest? 

Hint: 
- You will need to enable the `Google Places API`.
- You may find this page useful to learn about different findinging nearby places https://www.geeksforgeeks.org/python-fetch-nearest-hospital-locations-using-googlemaps-api/

In [37]:
import importlib
import os
import googlemaps

In [38]:
os.getcwd()

'/Users/almavelazquez/Documents/GitHub/PythonCamp2024/Day05/Lecture'

In [8]:
os.chdir('/Users/almavelazquez/Documents/GitHub/PythonCamp2024/Day05/Lecture')
imported_items = importlib.import_module('start_google')
gmaps = imported_items.client

whitehouse = '1600 Pennsylvania Avenue, Washington, DC'

embassies = [[38.917228,-77.0522365], 
	[38.9076502, -77.0370427], 
	[38.916944, -77.048739] ]


1. Which of these embassies is closest to the White House in meters? 

In [39]:
# save whitehouse geocode object
wh = gmaps.geocode(whitehouse)

##### Exploratory work
* Find distance to embassies

In [40]:
test_dist = gmaps.distance_matrix(wh[0]['geometry']['location'], embassies)

In [20]:
test_dist

{'destination_addresses': ['2341 Wyoming Ave NW, Washington, DC 20008, USA',
  '1601 Massachusetts Ave NW, Washington, DC 20036, USA',
  '2144 Wyoming Ave NW, Washington, DC 20008, USA'],
 'origin_addresses': ['1600 Pennsylvania Avenue NW, Washington, DC 20500, USA'],
 'rows': [{'elements': [{'distance': {'text': '4.3 km', 'value': 4299},
     'duration': {'text': '15 mins', 'value': 873},
     'status': 'OK'},
    {'distance': {'text': '3.0 km', 'value': 2957},
     'duration': {'text': '11 mins', 'value': 660},
     'status': 'OK'},
    {'distance': {'text': '3.9 km', 'value': 3943},
     'duration': {'text': '14 mins', 'value': 823},
     'status': 'OK'}]}],
 'status': 'OK'}

In [22]:
test_dist['rows'][0]['elements']

[{'distance': {'text': '4.3 km', 'value': 4299},
  'duration': {'text': '15 mins', 'value': 873},
  'status': 'OK'},
 {'distance': {'text': '3.0 km', 'value': 2957},
  'duration': {'text': '11 mins', 'value': 660},
  'status': 'OK'},
 {'distance': {'text': '3.9 km', 'value': 3943},
  'duration': {'text': '14 mins', 'value': 823},
  'status': 'OK'}]

* Each dictionary in this list corresponds to each embassy in the list

In [23]:
test_dist['rows'][0]['elements'][0]

{'distance': {'text': '4.3 km', 'value': 4299},
 'duration': {'text': '15 mins', 'value': 873},
 'status': 'OK'}

In [25]:
test_dist['rows'][0]['elements'][0]['distance']['value']

4299

##### Put it all together in a function

In [28]:
# define function to calculate distance to each embassy
def dist_finder(dest, origin = wh): # takes argument for destination (lat/lng), and origin with wh default
	
    # access origin latitude & longitude
    orig_lat_long = origin[0]['geometry']['location']
	
    # use distance API to calculate distance between the destination and origin lat/lngs
    dist = gmaps.distance_matrix(orig_lat_long, dest)

    # instantiate empty list
    meters=[]
    
    # for 1, 2, 3 of the embassies
    for i in range(3):
        # add the following to the empty list
        meters.append(float(dist['rows'][0]['elements'][i]['distance']['value']))
        
        return dest[meters.index(min(meters))]

# find closest embassy
closest = dist_finder(dest = embassies)

closest

[38.917228, -77.0522365]

What is the address? 

In [41]:
# address of closest
closest_address = gmaps.reverse_geocode(closest)[0]['formatted_address']
closest_address

'2341 Wyoming Ave NW, Washington, DC 20008, USA'

2. If I wanted to hold a morning meeting there, which cafes would you suggest for breakfast (best rating and closest)?

In [30]:
bfast = gmaps.places_nearby(closest, type = 'cafe', 
                            rank_by = "distance", 
                            keyword = 'breakfast')['results']

In [31]:
bfast

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 38.9237209, 'lng': -77.0523722},
   'viewport': {'northeast': {'lat': 38.92499947989272,
     'lng': -77.05114142010729},
    'southwest': {'lat': 38.92229982010728, 'lng': -77.05384107989272}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
  'icon_background_color': '#FF9E67',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
  'name': 'Open City',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 2736,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100676619916060645946">nakhun 41</a>'],
    'photo_reference': 'AelY_CuK4bgrUAA2B0ZLqHl9mGTyLacE57-UH0TnfntbmpXk3FUDlKlhHDhj8OxA-76xC86uuoPoSmB8Pah0GCyjdypWQDdEEJN_LaFkF4d-89VCdTLtFjbBWSaqfra-c4qM-tb5dOuVErNJLf6iOxK-8VtPTFOmEloVOs3TG-UssjTwwB76',
    'width': 3648}],
  'place_id': 'ChIJFaK2ddS3t4kR4LLXUHpoLC0',
  'plus_code': {'compound_co

In [33]:
len(bfast)

20

##### Closest

In [32]:
print('Closest breakfast place is {} at {}'.format(bfast[0]['name'], bfast[0]['vicinity'])) # closest

Closest breakfast place is Open City at 2331 Calvert St NW, Washington


In [34]:
# highest rated
rating = []
for i in range(len(bfast)):
	try:
		rating.append(bfast[i]['rating'])
	except:
		rating.append(0)
best_bfast = bfast[rating.index(max(rating))]

##### Best-rated

In [42]:
print('Best breakfast place is {} at {} with {}'.format(best_bfast['name'], 
                                                best_bfast['vicinity'], best_bfast['rating'])) # highest rated

Best breakfast place is Dolan Coffee at 3518 Connecticut Ave NW Unit D, Washington with 5


3. If I wanted to hold an upscale evening meeting there, which fancy bar would you suggest? 

In [43]:
# using a slightly different method than for bfast
bars = gmaps.places('bars near ' + '2341 Wyoming Ave NW, Washington, DC 20008, USA')

dollar = []
for i in range(0, len(bars)):
	try:
	    dollar.append(bars['results'][i]['price_level'])
	except:
	    dollar.append(0)
fancy_bar = bars['results'][dollar.index(max(dollar))]

print('Fanciest bar is {} at {}'.format(fancy_bar['name'], fancy_bar['formatted_address'])) # fanciest

Fanciest bar is Jack Rose Dining Saloon at 2007 18th St NW, Washington, DC 20009, United States
